In [1]:
import tkinter as tk
from tkinter import filedialog
import cv2 as cv
import numpy as np
import os
# import tensorflow as tf
# from keras.models import load_model
from PIL import ImageTk, Image

faces_classifier = cv.CascadeClassifier('face_detection.xml')

# # Load gender recognition model
# gender_model = load_model('gender_recognition.h5')
# gender_label_dict = {0: 'Female', 1: 'Male'}

# # Load emotion detection model
# emotion_model = load_model('emotion1_detection.h5')
# emotion_label_dict = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# # Load age detection model
# age_model = load_model('age_recognition.h5')

def detect_gender(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    faces = faces_classifier.detectMultiScale(gray)
    
    for (x, y, w, h) in faces:
        face_img = gray[y:y+w, x:x+w]
        resized = cv.resize(face_img, (60, 60)) / 255.0
        resized = np.reshape(resized, (1, 60, 60, 1))
        # result = gender_model.predict(resized)
        # label = np.argmax(result, axis=1)[0]

        # Change the color based on the gender
        color = (255, 0, 0) # if gender_label_dict[label] == 'Male' else (180, 105, 255)  # Blue for male, pink for female
        

        cv.rectangle(image, (x, y), ((x + w), (y + h)), color, 5)
        # cv.putText(image, gender_label_dict[label], (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    return image

# def detect_emotion(image):
#     gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
#     faces = faces_classifier.detectMultiScale(gray)
    
#     for (x, y, w, h) in faces:
#         face_img = gray[y:y+w, x:x+w]
#         resized = cv.resize(face_img, (48, 48)) / 255.0
#         resized = np.reshape(resized, (1, 48, 48, 1))
#         result = emotion_model.predict(resized)
#         label = np.argmax(result, axis=1)[0]
#         cv.putText(image, emotion_label_dict[label], (x, y + h + 30), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
#     return image

# def detect_age(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    faces = faces_classifier.detectMultiScale(gray)
    
    for (x, y, w, h) in faces:
        face_img = gray[y:y+w, x:x+w]
        resized = cv.resize(face_img, (60, 60)) / 255.0
        resized = np.reshape(resized, (1, 60, 60, 1))
        result = age_model.predict(resized)
        age = np.argmax(result, axis=1)[0]  # Get the age with the highest probability

        cv.putText(image, str(age), (x, y-30), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    return image

# Function to detect gender and emotion in a still image
def picCheck(image_path):
    # Load the image
    image = cv.imread(image_path)

    # Detect gender
    image_with_gender = detect_gender(image)
    
    # # Detect emotion
    # image_with_emotion = detect_emotion(image_with_gender)

    # # Detect age
    # image_with_age = detect_age(image_with_emotion)
    
    cv.imshow("Project of Accuracy of Facial Recognition", image_with_gender)  # Display the image with all the detections
    # Wait for Enter key press to take a screenshot
    key = cv.waitKey(0)
    cv.destroyAllWindows()
        
# Function to detect gender and emotion in real-time video
def vidCheck():
    # Open video capture
    video = cv.VideoCapture(0)

    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break

        # Detect gender
        frame_with_gender = detect_gender(frame)
        
        # # Detect emotion
        # frame_with_emotion = detect_emotion(frame_with_gender)
        
        # # Detect age
        # frame_with_age = detect_age(frame_with_emotion)
        
        cv.imshow("Project of Accuracy of Facial Recognition", frame_with_gender)  # Display the frame with all the detections
        if cv.waitKey(1) == ord('q')or cv.getWindowProperty("Project of Accuracy of Facial Recognition", cv.WND_PROP_VISIBLE) < 1:
            break
        
    
    video.release()
    cv.destroyAllWindows()
    


# Function to open a file dialog for selecting an image file
def select_picture():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        # Call picCheck function with the selected image file
        picCheck(file_path)

# Function to open the real-time video check
def real_time_check():
    # Call vidCheck function for real-time video check
    vidCheck()

# Function to display the about page
def show_about():
    about_text = "Project of Accuracy of Facial Recognition.\n\n" \
                 "Authors: Alakbarzade Yusif Ivan Stankevich Daniel Michalak Mevin Biju \n" \
                 "Version: 1.0"
    tk.messagebox.showinfo("About", about_text)


# Create the main window
window = tk.Tk()
window.title("Project of Accuracy of Facial Recognition")
window.geometry("800x600")

# Set window background color
window.configure(bg="#f0f0f0")

# Create a heading label
lbl_heading = tk.Label(window, text="Facial Recognition Application", font=("Arial", 24, "bold"), bg="#f0f0f0")
lbl_heading.pack(pady=20)

# Create buttons with styling
btn_select_picture = tk.Button(window, text="Select Picture", command=select_picture, font=("Arial", 14, "bold"),
                              bg="#4caf50", fg="white", padx=20, pady=10)
btn_select_picture.pack(pady=10)

btn_real_time_check = tk.Button(window, text="Real-time Check", command=real_time_check, font=("Arial", 14, "bold"),
                               bg="#4caf50", fg="white", padx=20, pady=10)
btn_real_time_check.pack(pady=10)

# Create the menu bar
menu_bar = tk.Menu(window)
window.config(menu=menu_bar)

# Create the File menu
file_menu = tk.Menu(menu_bar, tearoff=0)
file_menu.add_command(label="Select Picture", command=select_picture)
file_menu.add_command(label="Real-time Check", command=select_picture)
file_menu.add_command(label="Exit", command=window.quit)

# Create the Help menu
help_menu = tk.Menu(menu_bar, tearoff=0)
help_menu.add_command(label="About", command=show_about)

# Add menus to the menu bar
menu_bar.add_cascade(label="File", menu=file_menu)
menu_bar.add_cascade(label="Help", menu=help_menu)

# Run the main window event loop
window.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\mevin\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\mevin\AppData\Local\Temp\ipykernel_14604\1234460106.py", line 130, in real_time_check
    vidCheck()
  File "C:\Users\mevin\AppData\Local\Temp\ipykernel_14604\1234460106.py", line 102, in vidCheck
    frame_with_gender = detect_gender(frame)
                        ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mevin\AppData\Local\Temp\ipykernel_14604\1234460106.py", line 25, in detect_gender
    faces = faces_classifier.detectMultiScale(gray)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
cv2.error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\objdetect\src\cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'

Exception in Tkinter callback
Traceback (most recent 

In [3]:
import tkinter as tk
from tkinter import filedialog
import cv2 as cv
import numpy as np

faces_classifier = cv.CascadeClassifier('face_detection.xml')

def detect_faces(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    faces = faces_classifier.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        cv.rectangle(image, (x, y), ((x + w), (y + h)), (0, 255, 0), 5)
    
    return image

# Function to detect faces in a still image
def picCheck(image_path):
    image = cv.imread(image_path)
    image_with_faces = detect_faces(image)
    cv.imshow("Facial Recognition", image_with_faces)
    cv.waitKey(0)
    cv.destroyAllWindows()
        
# Function to detect faces in real-time video
def vidCheck():
    video = cv.VideoCapture(0)

    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break

        frame_with_faces = detect_faces(frame)
        cv.imshow("Facial Recognition", frame_with_faces)
        if cv.waitKey(1) == ord('q') or cv.getWindowProperty("Facial Recognition", cv.WND_PROP_VISIBLE) < 1:
            break
        
    video.release()
    cv.destroyAllWindows()

# Function to open a file dialog for selecting an image file
def select_picture():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        picCheck(file_path)

# Function to open the real-time video check
def real_time_check():
    vidCheck()

# Create the main window
window = tk.Tk()
window.title("Facial Recognition")
window.geometry("800x600")
window.configure(bg="#f0f0f0")

lbl_heading = tk.Label(window, text="Facial Recognition Application", font=("Arial", 24, "bold"), bg="#f0f0f0")
lbl_heading.pack(pady=20)

btn_select_picture = tk.Button(window, text="Select Picture", command=select_picture, font=("Arial", 14, "bold"),
                               bg="#4caf50", fg="white", padx=20, pady=10)
btn_select_picture.pack(pady=10)

btn_real_time_check = tk.Button(window, text="Real-time Check", command=real_time_check, font=("Arial", 14, "bold"),
                                bg="#4caf50", fg="white", padx=20, pady=10)
btn_real_time_check.pack(pady=10)

menu_bar = tk.Menu(window)
window.config(menu=menu_bar)

file_menu = tk.Menu(menu_bar, tearoff=0)
file_menu.add_command(label="Select Picture", command=select_picture)
file_menu.add_command(label="Real-time Check", command=select_picture)
file_menu.add_command(label="Exit", command=window.quit)

menu_bar.add_cascade(label="File", menu=file_menu)

window.mainloop()


In [1]:
pip install dlib


^C
Note: you may need to restart the kernel to use updated packages.


  Using cached dlib-19.24.2.tar.gz (11.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for dlib: filename=dlib-19.24.2-cp311-cp311-win_amd64.whl size=2854509 sha256=a4c02722dcb0780f35b84775410a9b411280b5e84e7c10c63e9babcad7f75858
  Stored in directory: c:\users\mevin\appdata\local\pip\cache\wheels\61\05\62\44b0bf18a0f8f9a0d65337b11237ecf12926d0d6e3807500bb
Successfully built dlib


In [8]:
#Antonio
import matplotlib.pyplot as plt
import cv2
import glob
import numpy as np

# Retrieve image file paths from a specified folder
def get_image_paths(folder_path):
    return glob.glob(f'{folder_path}/**/*.jpg', recursive=True)

# Preprocess image by resizing and normalizing pixel values
def preprocess_image(img):
    img = cv2.resize(img, (224, 224))
    return img / 255.0

# # Display both the original and processed images side by side
# def display_original_and_processed(original, processed):
#     plt.figure(figsize=(10,5))

#     plt.subplot(1, 2, 1)
#     plt.imshow(original)
#     plt.title('Original Image')

#     plt.subplot(1, 2, 2)
#     plt.imshow(processed)
#     plt.title('Processed Image')

#     plt.show()

# # Process images from the specified folder and return the array of processed images
# def process_images_from_folder(folder_path, images_to_display, total_images_to_process=None):
#     image_paths = get_image_paths(folder_path)
    
#     # If total_images_to_process is specified, limit the number of images to be processed
#     if total_images_to_process:
#         image_paths = image_paths[:total_images_to_process]
        
#     processed_images = []
    
#     for index, img_path in enumerate(image_paths):
#         img = cv2.imread(img_path)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#         processed_img = preprocess_image(img)
#         processed_images.append(processed_img)

#         # Display images up to the number specified by images_to_display
#         if index < images_to_display:
#             display_original_and_processed(img, processed_img)

#     return np.array(processed_images)

#Adam
#face detection
def face_detection(pic):
    face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    #pic = cv2.cvtColor(pic, cv2.COLOR_BGR2GRAY)
    facesdet = face_classifier.detectMultiScale(pic, scaleFactor = 1.05, minNeighbors = 6, minsize=(25,25))
    return facesdet

#boxes
def box_disp(pic, facesdet):
    for (x, y, a, b) in facesdet:
        cv2.rectangle(pic, (x, y), (x + a, y + b), (0, 255, 0), 4)

def age_disp(pic, facesdet, ages):
    for (x, y, a, b) in facesdet:
        cv2.rectangle(pic, (x, y), (x + a, y + b), (0, 255, 0), 4)
    return
    
    

# Process video feed from a file or the webcam
def process_video(video_path=None):
    cap = cv2.VideoCapture(0 if video_path is None else video_path)

    if not cap.isOpened():
        print("Error: Couldn't open the video stream or file.")
        return

    try:
        while True:
            
            ret, frame = cap.read()
            
            if not ret:
                print("Error: Couldn't read a frame from the video.")
                break
            
            processed_frame = preprocess_image(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            
            
            facesdet = face_detection(processed_frame)
            box_disp(processed_frame, facesdet)

            cv2.imshow('Processed Video Stream', (processed_frame * 255).astype(np.uint8))

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

# Example usage
#processed_images = process_images_from_folder('./imdb_crop', images_to_display=5, total_images_to_process=10)

process_video()
#Adam
#Capturing Video from the Webcam
# process_video()
#Processing a Video File
# process_video("example.mp4")

error: OpenCV(4.8.1) :-1: error: (-5:Bad argument) in function 'detectMultiScale'
> Overload resolution failed:
>  - 'minsize' is an invalid keyword argument for CascadeClassifier.detectMultiScale()
>  - 'minsize' is an invalid keyword argument for CascadeClassifier.detectMultiScale()
